In [5]:
import os
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import pandas as pd
import time
import requests 
import urllib.request

path= "C://Users//hjh05//Downloads//ugi//chromedriver.exe"

# DATA wiht crawling
### 유기동물보호관리시스템의 [공고] 내용 크롤링 코드 중 일부

In [12]:
#정보 저장용 데이터프레임 먼저 만들기
data = pd.DataFrame(columns = ["상태","공고번호","축종","품종","털색","성별","중성화","특징","관할보호센터명","보호장소"])
data

,상태,공고번호,축종,품종,털색,성별,중성화,특징,관할보호센터명,보호장소


In [13]:
#크롤링
driver=webdriver.Chrome(path)
driver.get('https://www.animal.go.kr/front/awtis/public/publicList.do?menuNo=1000000055#') #크롤링 사이트

source = driver.page_source
soup = BeautifulSoup(source, 'html.parser')
    
# 기간 설정
sdate = driver.find_element_by_xpath('//*[@id="searchSDate"]')
sdate.send_keys("002018-01-01")

edate = driver.find_element_by_xpath('//*[@id="searchEDate"]')
edate.send_keys("002021-02-28")

driver.find_element_by_css_selector('#search_button').click()

#현재 페이지 내 10개 정리하기

for i in range(1,11) :
    this_infos=[] #정보 받을 준비
    source = driver.page_source
    soup = BeautifulSoup(source, 'html.parser')
    
    #분양상태
    con = soup.select("#searchList > div.boardList > ul:nth-child(2) > li:nth-child(" +str(i) + ") > div.txt > dl:nth-child(7) > dd")
    con = con[0].text
    this_infos.append(con.strip())
    
    #상세보기
    xpath = '//*[@id="searchList"]/div[4]/ul[2]/li[' + str(i) + ']/div[1]/a'
    
    driver.find_element_by_xpath(xpath).click()
    source=driver.page_source
    soup=BeautifulSoup(source, 'html.parser')

    #동물정보
    for j in range(2,9):
        where = "#publicForm > div > table > tbody > tr:nth-child(" +str(j) + ") > td"
        info = soup.select(where)
        info = info[0].text
        info = info.strip()

        this_infos.append(info)


    #센터이름
    info = soup.select("#publicForm > div > table > tbody > tr:nth-child(16) > td")
    info = info[0].text
    info = info.strip()
    this_infos.append(info)        
    
    #보호장소
    info = soup.select("#publicForm > div > table > tbody > tr:nth-child(17) > td")
    info = info[0].text
    info = info.strip()
    this_infos.append(info)        
       
    data.loc[len(data)] = this_infos
    driver.back()
    
driver.close()

In [14]:
data

,상태,공고번호,축종,품종,털색,성별,중성화,특징,관할보호센터명,보호장소
0,공고중,전북-순창-2021-00005,개,믹스견,갈색,암컷,아니오,빨강색 목걸이 착용,원 동물병원,전라북도 순창군 순창읍 순화로 22 (순창읍)
1,보호중,전북-고창-2021-00055,개,믹스견,갈색,미상,아니오,새끼강아지,고창동물병원,전라북도 고창군 무장면 원금곡길 13
2,종료(자연사),전북-고창-2021-00054,개,믹스견,흰색,암컷,미상,자연사,고창동물병원,전라북도 고창군 무장면 원금곡길 13
3,보호중,경남-거제-2021-00187,개,믹스견,흰색,수컷,아니오,순함,거제시유기동물보호소,"경상남도 거제시 사등면 두동로1길 109 (사등면, 한국자원재생공사폐비닐적재장) ..."
4,보호중,경남-거창-2021-00044,개,믹스견,"검정색, 갈색",암컷,미상,5개월 추정,나래동물병원,"경상남도 거창군 거창읍 거창대로 13-2 (거창읍, 칠성슈퍼)"
5,종료(반환),경남-하동-2021-00077,개,라브라도 리트리버,흑색,수컷,아니오,순함,하동군유기동물보호소,경상남도 하동군 적량면 한옥정길 90 (적량면) 유기동물보호소
6,종료(반환),경남-하동-2021-00076,개,골든 리트리버,갈색,암컷,아니오,순함,하동군유기동물보호소,경상남도 하동군 적량면 한옥정길 90 (적량면) 유기동물보호소
7,종료(반환),경남-하동-2021-00075,개,말티즈,흰색,수컷,아니오,순함,하동군유기동물보호소,경상남도 하동군 적량면 한옥정길 90 (적량면) 유기동물보호소
8,보호중,경남-창녕-2021-00138,개,믹스견,노랑,수컷,아니오,야생 들개,창녕 유기동물보호소,경상남도 창녕군 고암면 창밀로 335-26 (고암면) 고암면 억만리 28
9,종료(자연사),경남-김해-2021-00204,개,믹스견,검정+갈색,암컷,미상,"1-191, 3개월추정, 허약",누리동물병원,부산광역시 해운대구 송정2로13번길 46 (송정동)
